### Cumulative ER Arrivals

We have a table showing the number of patients who arrive at the ER every day. How can we calculate the cumulative number of patients for each day, using SQL?

To answer this, let's first create some sample data. You'll find that pandas, datetime, and random are useful libraries for populating tabular structures with dummy data. 

For this exaample, we'll create a sample table with the day of year in one column and the number of ER arrivals in another column. 

In [1]:
import random
import pandas as pd

In [2]:
#help(random)
#help(datetime)

We'll model the number of arrivals as a random integer chosen between 50 and 150. This probably isn't realistic, data will probably be more normally distrubuted and fluctuate in response to external events, some of which are predicatable. But this will generate the sample data we need to write and test a SQL query.

In [3]:
def num_arrivals():
    return random.randint(50, 150)

Next up, we'll create a pandas dataframe holding the date for the next 100 days and a randomly generated number of patients arriving at the ER that day. Note that pandas a nice method for generating a series of dates. We'll generate the dates and number of arrivals as two lists, and assemble them into a dataframe using pandas

In [4]:
er_dates = []

er_arrivals = []
for i in range(100):
    er_arrivals.append(num_arrivals())

# use pandas
er_dates = pd.date_range(start='1/1/2020', periods=100)

In [8]:
df = pd.DataFrame({'er_date':er_dates, 'er_arrivals':er_arrivals})

In [9]:
df.head(10)

,er_date,er_arrivals
0,2020-01-01,60
1,2020-01-02,63
2,2020-01-03,111
3,2020-01-04,143
4,2020-01-05,85
5,2020-01-06,115
6,2020-01-07,132
7,2020-01-08,146
8,2020-01-09,56
9,2020-01-10,84


Alright! We're ready to write some SQL. 

Rather than building a database, I'll use a module, pandasql, that will allow us to run SQL code directly agaainst a pandas dataframe as if it were a table in a relational database, using SQLite syntax. 

In [10]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

Try it out! 

I recommend trying this out before jumping straight to the answer. There's value in sticking with it to the point of mild frustration :) Even if you end up looking at the answer, the time you spend puzzing over it will help you remember the solution and apply it to a related problem in the future.

Hint - One solution to this problem involves a self-join - Remember, you're not limited to joining table on an equality operator! Think about how you'd use an inequality along with a self-join...

Here's an example of using a self-join to get all the er_arrivals up to each date in the system. 

In [11]:
pysqldf("""
SELECT 
    date(a.er_date), date(b.er_date), b.er_arrivals
FROM 
    df a
JOIN 
    df b 
ON 
    a.er_date >= b.er_date
""").head(20)

,date(a.er_date),date(b.er_date),er_arrivals
0,2020-01-01,2020-01-01,60
1,2020-01-02,2020-01-01,60
2,2020-01-02,2020-01-02,63
3,2020-01-03,2020-01-01,60
4,2020-01-03,2020-01-02,63
5,2020-01-03,2020-01-03,111
6,2020-01-04,2020-01-01,60
7,2020-01-04,2020-01-02,63
8,2020-01-04,2020-01-03,111
9,2020-01-04,2020-01-04,143


#### Solution 1: Use SQL!
    
You can modify the query above, with an aggregation, to get the cumulative value for each date. 

In [22]:
sql = """
SELECT 
    date(a.er_date) as er_date, 
    SUM(b.er_arrivals) as er_cumulative 
FROM
    df a
JOIN 
    df b 
ON
    a.er_date >= b.er_date 
GROUP BY 
    a.er_date 
ORDER BY 
    a.er_date ASC
"""

In [23]:
df.columns

Index(['er_date', 'er_arrivals'], dtype='object')

In [24]:
pysqldf(sql)

,er_date,er_cumulative
0,2020-01-01,60
1,2020-01-02,123
2,2020-01-03,234
3,2020-01-04,377
4,2020-01-05,462
5,2020-01-06,577
6,2020-01-07,709
7,2020-01-08,855
8,2020-01-09,911
9,2020-01-10,995


#### Solution 2: Use pandas

Pandas has a cumulative sum method. 

In [27]:
df['er_arrivals'].cumsum()

0       60
1      123
2      234
3      377
4      462
5      577
6      709
7      855
8      911
9      995
10    1068
11    1165
12    1297
13    1361
14    1482
15    1534
16    1653
17    1741
18    1836
19    1939
20    1998
21    2128
22    2209
23    2277
24    2359
25    2425
26    2506
27    2573
28    2716
29    2793
      ... 
70    6977
71    7043
72    7166
73    7255
74    7314
75    7378
76    7484
77    7592
78    7726
79    7856
80    7906
81    8046
82    8118
83    8254
84    8305
85    8381
86    8464
87    8577
88    8664
89    8777
90    8828
91    8903
92    8995
93    9106
94    9218
95    9284
96    9412
97    9528
98    9645
99    9697
Name: er_arrivals, Length: 100, dtype: int64

### Exercise

In this workbook, we generated a sample table with the total number of ER arrivals for each date, and used this to calculate a cumulative sum for each date. 

As an follow up exercise, how would you model the arrival of each patient? Try logging the arrival date for each patient, and use this table to generate a cumulative sum of total patient arrivals in the ER for each day in your date range. 